## 03/04/2016 - Reading the parsed chromosome from ExAC and map the Esembl IDs to Hugo Symbols
This is an old version of parsing the chromosome files that includes mapping to Hugo symbols.

This mapping was needed for the first hmmer results obtained from Shilpa.

The second hmmer results already contain Ensembl id, so such mapping isn't needed

In [95]:
import pandas as pd
import numpy as np
from biomart import BiomartServer
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [97]:
#Getting the list of chromosome files
curr_dir = !pwd
my_path = curr_dir[0]+"/parsed/"
filename = "parsed_chrom"
chrom_files = !ls $my_path

In [98]:
#Importing Biomart database
server = BiomartServer("http://grch37.ensembl.org/biomart")
ens_genes = server.datasets[u'hsapiens_gene_ensembl']

In [109]:
for filename in chrom_files:
    #Reading the chromosomel file
    chrom_csv = pd.read_csv(my_path+filename, sep='\t', index_col=0)
    chrom_csv.fillna('', inplace=True)
    
    #Getting all unique Esembl IDs
    ensmbl_ids = ((chrom_csv["gene"]).unique()).tolist()
    
    #Creating a dictionary of ensembl id mapped to hugo symbol
    ens_hugo_dict = {}
    not_found = []
    for ens in ensmbl_ids:
        if ens.startswith("ENSG") == False:
            not_found.append(ens)
            continue

        #Querying the Biomart database to get number of matches
        num = ens_genes.count({
            'filters': {
                    'ensembl_gene_id': ens
            },
            'attributes': [
                    'hgnc_symbol'
            ]
        })

        #In case no match is found
        if (num == 0):
            not_found.append(ens)
            continue

        if (num > 1):
            print "more than 1 match for "+ens
            print num

        #In case of at least one match: querying the Biomart database
        response = ens_genes.search({
            'filters': {
                    'ensembl_gene_id': ens
            },
            'attributes': [
                    'hgnc_symbol'
            ]
        })

        #Reading the query result
        for line in response.iter_lines():
            line = line.decode('utf-8')
            hugo_sym = (line.split("\t")[0]).encode('ascii','ignore')
            break

        #Updating the dictionary
        ens_hugo_dict[ens] = hugo_sym
        
    #Create a mapping of Ensembl-Hugo and save to data-frame
    ensmbl_col = []
    hugo_col = []
    for ens in ens_hugo_dict.keys():
        ensmbl_col.append(ens)
        hugo_col.append(ens_hugo_dict[ens])
    mapping = pd.DataFrame({"Ensembl_gene_id": ensmbl_col, "Hugo_symbol": hugo_col})
    
    #Save to file
    chrom_number = filename[filename.find("chrom"):-4]
    mapping.to_csv(curr_dir[0]+"/ensembl-hugo_map/ids_map_"+chrom_number+".csv", sep='\t')
    
    print "finished "+chrom_number

finished chrom10
finished chrom11
finished chrom12
finished chrom13
finished chrom14
finished chrom15
finished chrom16
finished chrom17
finished chrom18
finished chrom19
finished chrom1
finished chrom20
finished chrom21
finished chrom22
finished chrom2
finished chrom3
finished chrom4
finished chrom5
finished chrom6
finished chrom7
finished chrom8
finished chrom9
finished chromX
finished chromY


/home/anat/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
#Creating a dictionary of ensembl id mapped to hugo symbol
ens_hugo_dict = {}
not_found = 0
for ens in ensmbl_ids:
    if ens.startswith("ENS") == False:
        not_found += 1
        continue
    
    #Querying the Biomart database to get number of matches
    num = ens_genes.count({
        'filters': {
                'ensembl_gene_id': ens
        },
        'attributes': [
                'hgnc_symbol'
        ]
    })
    
    #In case no match is found
    if (num == 0):
        print "No match for "+ens
        not_found += 1
        continue
    
    if (num > 1):
        print "more than 1 match for "+ens
        print num
    
    #In case of at least one match: querying the Biomart database
    response = ens_genes.search({
        'filters': {
                'ensembl_gene_id': ens
        },
        'attributes': [
                'hgnc_symbol'
        ]
    })
    
    #Reading the query result
    for line in response.iter_lines():
        line = line.decode('utf-8')
        hugo_sym = (line.split("\t")[0]).encode('ascii','ignore')
        break
    
    #Updating the dictionary
    ens_hugo_dict[ens] = hugo_sym

In [47]:
chrom_csv[chrom_csv["gene"] == "2349"]

,chrom,pos,id,ref,alt,qual,filter,AC,AF,AN,gene,conseq,prot_pos,amino_acids,codons,SWISSPROT,SIFT,domains,clin_sig
263,1,664678,.,G,A,562.11,VQSRTrancheSNP99.60to99.80,8,7.672e-04,10428,2349,31,1,0,0,2,0,0,0;GQ_HIST=51
477764,1,146253158,.,C,G,199.75,VQSRTrancheSNP99.80to99.90,2,5.377e-05,37196,2349,2299,456,231,134,1,0,0,0;GQ_HIST=1472


In [82]:
#Create a mapping of Ensembl-Hugo
ensmbl_col = []
hugo_col = []
for ens in ens_hugo_dict.keys():
    ensmbl_col.append(ens)
    hugo_col.append(ens_hugo_dict[ens])
mapping = pd.DataFrame({"Ensembl_gene_id": ensmbl_col, "Hugo_symbol": hugo_col})
mapping.to_csv(curr_dir[0]+"/ensembl-hugo_map/ids_map_1.csv", sep='\t')

In [80]:
mapping

,Ensembl_gene_id,Hugo_symbol
0,ENSG00000064703,DDX20
1,ENSG00000159363,ATP13A2
2,ENSG00000196482,ESRRG
3,ENSG00000117215,PLA2G2D
4,ENSG00000086015,MAST2
5,ENSG00000142675,CNKSR1
6,ENSG00000142676,RPL11
7,ENSG00000142677,IL22RA1
8,ENSG00000215883,CYB5RL
9,ENSG00000179743,FBLIM1
